In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [4]:
import platform 
from pyspark.sql import SparkSession
    
if platform.system() == 'Windows':
    path = 'C:'
    spark = (SparkSession.builder.master("local[*]")
        .appName("TesteHive")
        .config("hive.metastore.uris", "thrift://192.168.15.4:9083")
        .config("spark.sql.warehouse.dir","hdfs://localhost:9000/users/hive/warehouse")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.sql.adaptive.enabled", "true")
        .enableHiveSupport()
        .getOrCreate())
else:
    try:
        spark.stop()
    except:
        pass
    path = '/mnt/c'
    spark = (SparkSession.builder.appName("MyApp")
            .config("hive.metastore.uris", "thrift://192.168.15.4:9083")
            .config("spark.sql.warehouse.dir","hdfs://localhost:9000/users/hive/warehouse")
            .config("spark.jars.packages","io.delta:delta-core_2.12:2.4.0")
            .config("spark.driver.memory","2g")
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            .enableHiveSupport()
            .getOrCreate())
    
    # spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark

:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2c3711e0-853c-4b32-94c4-8590383e7966;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.4.0/delta-core_2.12-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.4.0!delta-core_2.12.jar (1340ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/2.4.0/delta-storage-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;2.4.0!delta-storage.jar (264ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (294ms)
:: resolution report :: resolve 3717ms :: artifacts dl 1902m

In [5]:
spark.range(10).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [10]:
!hdfs dfs -put ingestao.ipynb /user

24/06/03 01:35:12 WARN HiveClientImpl: HiveClient got thrift exception, destroying client and retrying (0 tries remaining)
org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
	at org.apache.hadoop.hive.ql.metadata.Hive.getDatabase(Hive.java:1567)
	at org.apache.hadoop.hive.ql.metadata.Hive.databaseExists(Hive.java:1552)
	at org.apache.spark.sql.hive.client.Shim_v0_12.databaseExists(HiveShim.scala:609)
	at org.apache.spark.sql.hive.client.HiveClientImpl.$anonfun$databaseExists$1(HiveClientImpl.scala:406)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at org.apache.spark.sql.hive.client.HiveClientImpl.$anonfun$withHiveState$1(HiveClientImpl.scala:303)
	at org.apache.spark.sql.hive.client.HiveClientImpl.liftedTree1$1(HiveClientImpl.scala:234)
	at org.apache.spark.sql.hive.client.HiveClientImpl.retryLocked(HiveClientImpl.scala:233)
	at org.apache.spark.

In [6]:
spark.sql('''
show tables

''').show()

24/06/03 01:34:27 WARN metastore: Failed to connect to the MetaStore Server...
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [3]:
spark.sql('''

select 

--substring(UF_zi,1,2) as UF,
--count (distinct ano_cmpt),
--count (distinct mes_cmpt),
count (distinct N_AIH,DT_SAIDA,IDENT),
count(*)

from bronze.rd_sih

--group by 1
--order by 1
''').show(30)

+--------------------------------------+--------+
|count(DISTINCT N_AIH, DT_SAIDA, IDENT)|count(1)|
+--------------------------------------+--------+
|                                299648|  299648|
+--------------------------------------+--------+



In [4]:
df = spark.read.csv('/DataSus/rd/csv',sep=";",header=True)
df.createOrReplaceTempView("test")

spark.sql('''

select 

--substring(UF_zi,1,2) as UF,
--count (distinct ano_cmpt),
--count (distinct mes_cmpt),
count (distinct N_AIH,DT_SAIDA,IDENT),
count(*)

from test

--group by 1
--order by 1
''').show(30)

+--------------------------------------+--------+
|count(DISTINCT N_AIH, DT_SAIDA, IDENT)|count(1)|
+--------------------------------------+--------+
|                                299648|  345124|
+--------------------------------------+--------+



In [20]:
df = pd.DataFrame({'name': ['Raphael', 'Donatello'],
                   'mask': ['red', 'purple'],
                   'weapon': ['sai', 'bo staff']})


In [25]:

df.to_csv('df.csv')